#### Здравствуйте

Продолжаю улучшать свои навыки и знания.

**Предлагаю ознакомиться с мини проектом "SQL в Pandas".**

В таблице user_actions сохраняются действия покупателей в интернет магазине, структура таблицы

* user_id    - id пользователя
* product_id - id товара
* action     - действие, просмотр товара, сохранение в корзину, покупка
* time       - время совершения события
* date       - дата события

Необходимо для каждого дня рассчитать, какой процент просмотров товаров завершился покупкой

In [1]:
import pandas as pd
from pandasql import sqldf 
import numpy as np

In [2]:
#  Создам базу данных user_actions в которой сгенирую действия покупателей в интернет магазине,
# данные логически верные, но не подойдут для выявления паттернов пользователей.

# Cтруктура таблицы:
#   user_id - id пользователя
#   product_id - id товара
#   action - действие, просмотр товара, сохранение в корзину, покупка
#   time - время совершения события

In [3]:
all_user_ids = np.arange(1, 1001)
all_product_ids = np.arange(1, 101)
n = 10000

# Сгенерировал масив 1000 пользователей и  100 товаров
# n - к-во строк

In [5]:
user_ids    = np.random.choice(all_user_ids, n)
product_ids = np.random.choice(all_product_ids, n)

# с помощью функции random.choice сгенерирова  id и событий связанных с продуктами(с повторениями)

start_date = pd.to_datetime('2022-01-01')

# сгенерировал временную шкалу, выбрав дату старта

times = pd.date_range(start_date, periods=n, freq='1min')

# указал интервал времени между событиями

In [6]:
user_actions = pd.DataFrame({'user_id': user_ids, 
                             'product_id': product_ids, 
                             'time': times})

#создал Dataframe из списка, испульзуя списки словарей в качестве входных данных, имена столбцов в качестве ключей.

In [7]:
user_actions['action'] = 'view'
# добавил колонку action 

In [8]:
user_actions.head()

,user_id,product_id,time,action
0,151,60,2022-01-01 00:00:00,view
1,553,7,2022-01-01 00:01:00,view
2,150,67,2022-01-01 00:02:00,view
3,291,39,2022-01-01 00:03:00,view
4,186,34,2022-01-01 00:04:00,view


In [9]:
def generate_funel_actions(user_id, product_id, time):
    to_cart = 0.2
    to_purchase = 0.4
    
    df = pd.DataFrame()
    
    if np.random.binomial(1, to_cart, 1)[0]:
        df = pd.DataFrame({
                           'user_id'   : user_id, 
                           'product_id': product_id, 
                           'time'      : time + pd.Timedelta(5, unit='s'), 
                           'action'    : 'add to cart'}, index=[0])
        
        if np.random.binomial(1, to_purchase, 1)[0]:
            df_purchase = pd.DataFrame({
                           'user_id'   : user_id, 
                           'product_id': product_id, 
                           'time'      : time + pd.Timedelta(10, unit='s'), 
                           'action'    : 'purchase'}, index=[0])
    
            df = df.append(df_purchase)
    return df

# функция реализует алгоритм подбрасывания монетки 
# с вероятностью 0.2 пользователь помещает товар в корзину, после просмотра
# с вероятностью 0.4 пользователь совершает покупку товаров которые он поместил в корзину
# np.random.binomial возвращает массив на выходе, [0] -мы берем первый и единственный параметр, [1]-у функции означает размерность,
# по условию биномиального распределения, функция создает запись о добавлении товара в корзину и запись времени этого события с интервалом на 5 секунд относительно просмотра
# генерирую еще одно дейстивие, пользователь совершает покупку товара из козины с вероятностью 0.4 и создается запись действия с временным интервалом в 10 секунд
# функция возвращается в начало df

In [10]:
for i, row in user_actions.iterrows():
    user_df = generate_funel_actions(row['user_id'], row['product_id'], row['time'])
    user_actions = user_actions.append(user_df)
    
#  concat() подходит для обьединения df 
# iterrows() применить функцию к строкам 

In [11]:
user_actions = user_actions.sort_values('time')

In [12]:
user_actions['date'] = user_actions.time.dt.date

In [13]:
user_actions.head(15)

,user_id,product_id,time,action,date
0,151,60,2022-01-01 00:00:00,view,2022-01-01
1,553,7,2022-01-01 00:01:00,view,2022-01-01
2,150,67,2022-01-01 00:02:00,view,2022-01-01
3,291,39,2022-01-01 00:03:00,view,2022-01-01
4,186,34,2022-01-01 00:04:00,view,2022-01-01
5,133,53,2022-01-01 00:05:00,view,2022-01-01
0,133,53,2022-01-01 00:05:05,add to cart,2022-01-01
6,55,100,2022-01-01 00:06:00,view,2022-01-01
0,55,100,2022-01-01 00:06:05,add to cart,2022-01-01
0,55,100,2022-01-01 00:06:10,purchase,2022-01-01


In [13]:
# Решение задачи
# Для каждого дня рассчитайтаю, какой процент просмотров товаров завершился покупкой

In [14]:
q = """SELECT date,
              views, 
              carts, 
              purchases, 
              100 * purchases / views as purchase_percantage 
       FROM (
           SELECT date, 
            count(case when action = 'view'        then 1 else NULL end) as views, 
            count(case when action = 'add to cart' then 1 else NULL end) as carts, 
            count(case when action = 'purchase'    then 1 else NULL end) as purchases 
           FROM user_actions 
           GROUP BY date);"""

# Выражение CASE – условный оператор языка SQL. Данный оператор позволяет осуществить проверку условий и возвратить в зависимости от выполнения того или иного условия тот или иной результат. 
# Выражением CASE, позволяет включить условные выражения в запрос.
# Функция, COUNT (*) возвращает количество строк в указанной таблице с учетом повторяющихся строк.
# можно разделить количество покупок на количество просмотров и умножить 100.

In [15]:
sqldf(q)

,date,views,carts,purchases,purchase_percantage
0,2022-01-01,1440,278,101,7
1,2022-01-02,1440,282,108,7
2,2022-01-03,1440,290,117,8
3,2022-01-04,1440,266,105,7
4,2022-01-05,1440,290,122,8
5,2022-01-06,1440,298,123,8
6,2022-01-07,1360,283,104,7


##### Источники информации, применяемые в процессе решения

* https://numpy.org/doc/stable/reference/random/generated/numpy.random.binomial.html
* https://www.cyberforum.ru/python-tasks/thread2883132.html
* https://www.dmitrymakarov.ru/python/random-11/
* https://all-python.ru/osnovy/funktsii.html
* https://pythonru.com/biblioteki/vozmozhnosti-obektov-index-v-pandas-pd-3
* https://www.stackfinder.ru/questions/27644617/difference-between-n-and-size-parameters-in-np-random-binomialn-p-size-1000
* https://docs.sqlalchemy.org/en/14/index.html

##### SQL

* https://pypi.org/project/pandasql/
* https://oracleplsql.ru/if-else-statement-sql-server.html
* https://habr.com/ru/post/255825/


# Аналитическая записка

### Итоги 

При выполнении данного задания, мной создана база данных пользователей, произведена генерация данных числами, представляющими биномиальное распределение. Данные в базе логичны и воспроизводят структуру действий покупателей интернет магазина. Применив данную базу данных, решил поставленную задачу и произвел расчет процентов просмотра товаров завершившихся покупкой, для каждого дня данной выборки.


                                                                                                           С уважением,
                                                                                                          Осокин Максим